This script generates code smells using an adapted version of the Pysmell software tool over the buggy data


In [ ]:
import os
import subprocess
from subprocess import Popen
import pandas as pd
import numpy as np
import glob
import sys
from pathlib import Path
#getting full path to import adapted version of the Pysmell software tool
cwd = os.getcwd()
path = Path(cwd).parent.parent
toolpath = str(path) + r'\tools\tmpPysmell'
astpath = str(path) + r'\tools\tmpPysmell\ast'
sys.path.append(toolpath)
sys.path.append(astpath)
from pysmell import *




Creates the dataframe where all code smells will be stored, dataframe will be converted to a CSV file later on

In [ ]:
bug_smells = pd.DataFrame(columns =['bug_id','bug_type','project_full_name','file_name','function_name','code_smell_metric','code_smell_metric_value'])

In [ ]:
bug_smells.head()


Code_smells_to_csv function - Returns a dataframe with a row having bug_id,bug_type,file_name and smells to be added to bug_smells CSV

In [ ]:
bugs = pd.read_csv(r'..\..\subjects\data\generated\bugs-in-quantum-computing-platforms.csv', sep=',')

In [ ]:
bugs.shape

In [ ]:
bugs.head()

In [ ]:
# Converting series to String

bugs['fix_commit_hash']= pd.Series(bugs['fix_commit_hash'], dtype="string")
bugs['bug_id']= pd.Series(bugs['bug_id'], dtype="string")
bugs['bug_type']= pd.Series(bugs['bug_type'], dtype="string")
bugs['project_full_name']= pd.Series(bugs['project_full_name'], dtype="string")

In [ ]:
length = bugs.shape[0]
j = 0

for i in range(length):
    sha1 = (bugs['fix_commit_hash'].iloc[i])
    repo_url = 'https://github.com/' + bugs['project_full_name'].iloc[i] +'.git'
    sub_repo_url = bugs['project_full_name'].iloc[i].split('/')
    print(sha1)
    print(repo_url)
    print(sub_repo_url[1])
    
    ### Clone Repository
    process = subprocess.Popen(['git','clone',repo_url],cwd = r'..\..\subjects\tmp-repositories', stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output, errors = process.communicate()

    ### Checkout to the fixed version
    process2 = subprocess.Popen(['git','checkout',sha1+'~1'],cwd = r'..\..\subjects\tmp-repositories' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output2, errors = process2.communicate()
    
    ### Obtain only the modified files
    
    process3 = subprocess.Popen(['git','diff-tree','--no-commit-id','--name-only','-r',sha1 + '~1'],cwd = r'..\..\subjects\tmp-repositories' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output3, errors = process3.communicate()
    #print(output3.splitlines())
    
    ### todos os ficheiros python de um dado repo
    pyPath = r'..\..\subjects\tmp-repositories' + '\\' + sub_repo_url[1]
    #ficheiros = glob.glob(pyPath + '/**/*.py',recursive=True)
    
    print('Hash = ' + sha1)
    
    for ficheiro in output3.splitlines():
        
        #print(ficheiro)
        idd = str(bugs['bug_id'].iloc[i])
        tipo = bugs['bug_type'].iloc[i]
        ficheiro_final = pyPath + '\\' + ficheiro
        print(ficheiro_final)
                
        try:
            bug_smells = bug_smells.append(Code_smells_to_csv(idd,tipo,ficheiro_final,sub_repo_url[1]),ignore_index = True)
          
           
        except:
            print("An exception occurred. The file is not a python file")
        

In [ ]:
bug_smells.head()

In [ ]:
bug_smells.shape

In [ ]:
bug_smells.to_csv(r'..\data\generated\code-smell-metrics-for-buggy-data.csv', index=False)

In [ ]:
bug_smells.describe()

In [ ]:
print(bug_smells['project_full_name'].unique())